In [1]:
import os
import tempfile
from tqdm import tqdm
from collections import defaultdict
from Bio.Seq import Seq
from Bio.Align.Applications import MafftCommandline
from pymsaviz import MsaViz

In [ ]:
cwd='/path/to/consensus/sequences'

In [3]:
seq_dict = {}
for file in tqdm(os.listdir(cwd)):
    try:
        with open(os.path.join(cwd,file), 'r') as f:
            for line in f:
                if 'jgi.p|Pucstr1|474\n' in line: # for PST130_P495001
                # if 'jgi.p|Pucstr1|9243\n' in line: # for PST130_09225
                # if 'jgi.p|Pucstr1|20937\n' in line: # for PST130_06515
                # if 'jgi.p|Pucstr1|11781\n' in line: # for PST130_14067
                    isol=file.split('_')[-4]
                    seq_dict[isol]=next(f).strip()
                    break
    except:
        continue   

100%|██████████| 414/414 [02:22<00:00,  2.90it/s]


In [4]:
total_count = 0
total_cov = 0

for isol,seq in seq_dict.items():
    amb = seq.count('?')
    cov = (len(seq)-amb)/len(seq)
    if cov > 0:
        total_cov += (1-cov)
        total_count += 1

print(total_cov/total_count)


0.03995134169872601


In [5]:
uniq_seqs = {}
seq_counts = defaultdict(int)
hap_isols = {}
count = 0

for isol, seq in tqdm(seq_dict.items()):
    if seq not in uniq_seqs.values():
        count += 1
        uniq_seqs[f'H{count}'] = seq
        hap_isols[seq] = [isol]
    seq_counts[seq] += 1
    if isol not in hap_isols[seq]:
        hap_isols[seq].append(isol)

for seq, count in seq_counts.items():
    for hap, seq2 in uniq_seqs.items():
        if seq == seq2:
            uniq_seqs[hap] = seq, count


100%|██████████| 413/413 [00:00<00:00, 381636.39it/s]


In [6]:
haps = {}
new_index = 0

if os.path.exists('haplotypes.fna'):
    os.remove('haplotypes.fna')
if os.path.exists('haplotypes.info'):
    os.remove('haplotypes.info')

for hap,(seq,count) in uniq_seqs.items():
    amb=seq.count('?')
    amb_pct = amb / len(seq)
    cov=(len(seq)-amb)/len(seq)
    if amb_pct > 0:
        continue
    new_index+=1
    for hapseq, isol in hap_isols.items():
        if hapseq == seq:
            with open('haplotypes.info', 'a') as fout:
                print(f'H0{new_index}: {isol}', file=fout)
    print(f'>H0{new_index}({count})\n{seq}')
    haps[f'H0{new_index}']=seq
    with open('haplotypes.fna', 'a') as f:
        f.write(f'>H0{new_index}({count})\n{seq}\n')

>H01(263)
ATGAAGACTTCAATCCAGTTGGTTATTGTCATCGGCACACTCATCACAGCTTGTGCAGGATTGATTTCAACCATCACACATGAGAACATCGAATATAATATCGTCAACGCACTAACCGACCTTCATGGGGTCGATGGTCCACTCCATAATTTTGTCTCCGATAATGCAGTTCGAATTGGACACACCAATATTGCAAATGATTCACCCAGAAAGGTCGGCTTCTACACCAACTTGCCTGGAAGCACATATGTATTGTGGTTAGATCCCGGGGAAAAAGGTACTCTACATTTCAACAAGGATTATCCCTGGGTATTGGCGTCTGGAGAGCCTCGAGCCAATTTAAACAAGGAGATATACGAGATCCTTGTACCCCAGGTAATCAACATGAACGAGGACGTTTTGGGCATAATGACTTTTCTTCGCTCTTTGCGAGATTAA
>H02(6)
ATGAAGACTTCAATCCAGTTGGTTATTGTCATCGGCACACTCATCACAGCTTGTGCAGGATTGATTTCAACCATCACACATGAGAACATCGAATATAATATCGTCAACGCACTAACCGACCTTCATGGGGTCGATGGTCCACTCCATAATTTTGTCTCCGATAATGCAGTTCGAATTRGACACACCAATATTGCAAATGATTCACCCAGAAAGGTCGGCTTCTACACCAACTTGCCTGGAAGCACATATGTATTGTGGTTAGATCCCGGGGAAAAAGGTACTCTACATTTCAACAAGGATTATCCCTGGGTATTGGCGTCTGGAGAGCCTCGAGCCAATTTAAACAAGGAGATATACGAGATCCTTGTACCCCAGGTAATCAACATGAACGAGGACGTTTTGGGCATAATGACTTTTCTTCGCTCTTTGCGAGATTAA
>H03(1)
ATGAAGACTTCAATCCAGTTGGTTATTGTCATCGGCACACTCATCACAGCTTGTGCAGGATTGATTTCAACCATCACACATGAGAACATCGAATAT

In [7]:
isof = {}
isof_count = 0

if os.path.exists('isoforms.fna'):
    os.remove('isoforms.fna')

for hap,seq in haps.items():
    aa=Seq(seq).translate()
    if aa not in isof.values():
        isof_count+=1
        print(f'I0{isof_count}\t{aa}')
        isof[f'I0{isof_count}']=aa
        with open('isoforms.fna', 'a') as f:
            f.write(f'>I0{isof_count}\n{aa}\n')

I01	MKTSIQLVIVIGTLITACAGLISTITHENIEYNIVNALTDLHGVDGPLHNFVSDNAVRIGHTNIANDSPRKVGFYTNLPGSTYVLWLDPGEKGTLHFNKDYPWVLASGEPRANLNKEIYEILVPQVINMNEDVLGIMTFLRSLRD*
I02	MKTSIQLVIVIGTLITACAGLISTITHENIEYNIVNALTDLHGVDGPLHNFVSDNAVRIXHTNIANDSPRKVGFYTNLPGSTYVLWLDPGEKGTLHFNKDYPWVLASGEPRANLNKEIYEILVPQVINMNEDVLGIMTFLRSLRD*
I03	MKTSIQLVIVIGTLITACAGLISTITHENIEYNIVNALTDLHGVDGPLHNFVSDNXVRIGHTNIANDSPRKVGFYTNLPGSTYVLWLDPGEKGTLHFNKDYPWVLASGEPRANLNKEIYEILVPQVINMNEDVLGIMTFLRSLRD*
I04	MKTSIQLVIVIGTLITACAGLISTITHENIEYNIVNALTDLHGVDGPLHNFVSDNAVRIRHTNIANDSPRKVGFYTNLPGSTYVLWLDPGEKGTLHFNKDYPWVLASGEPRANLNKEIYEILVPQVINMNEDVLGIMTFLRSLRD*


In [8]:
from shutil import copyfile

def align_seqs(sampleid, sample_dict):
    with tempfile.NamedTemporaryFile(mode="w+", delete=False) as temp_input:
        for sample,qseq in sample_dict.items():
            temp_input.write(f">{sample}\n{qseq}\n")
            temp_input.flush()

        temp_output_name = temp_input.name + ".aln"
        mafft_cline = MafftCommandline(input=temp_input.name)
        stdout, stderr = mafft_cline()
        with open(temp_output_name, "w") as temp_output:
            temp_output.write(stdout.upper())


    copyfile(temp_output_name, f"{sampleid}.aln")
    mv = MsaViz(temp_output_name, show_count=True)
    mv.savefig(f"{sampleid}.png")
    

In [9]:
align_seqs('haplotypes', haps)